# 使用Tensorflow (低级API)构建CNN对cifar10数据进行分类

我们使用tf.nn的api对cifar10进行分析。
构建如下结构的cnn，来对构建一个cifar10的分类模型。

input: (None, 32, 32, 3)

conv layer1: 

kernel size = (5, 5), filters = 32, strides = (1, 1, 1, 1)

maxpooling layer1:

strides = (1, 2, 2, 1)

conv layer2:

kernel size = (5, 5), filters = 64, strides = (1, 1, 1, 1)

maxpooling layer2:

strides = (1, 2, 2, 1)

fc layer:

filters=1024 

softmax output layer
filters = 10

# 导入数据

In [ ]:
%matplotlib inline

import os
import numpy as np
import tensorflow as tf


DATA_PATH = './data/cifar10'
x_train = np.load(DATA_PATH + os.sep + 'x_train.npy')
y_train = np.load(DATA_PATH + os.sep + 'y_train.npy')
x_test = np.load(DATA_PATH + os.sep + 'x_test.npy')
y_test = np.load(DATA_PATH + os.sep + 'y_test.npy')

x_train = x_train / 255
x_test = x_test / 255

In [ ]:
from matplotlib import pyplot as plt

images_and_labels = list(zip(x_train, y_train))
plt.figure(figsize=(10,10), dpi=100)
for index, (image, label) in enumerate(images_and_labels[:32]):
    plt.subplot(4, 8, index + 1)
    # 不显示坐标轴
    plt.axis('off')
    plt.imshow(image)
    plt.title('%i' % label)
plt.show()

In [ ]:
# one-hot coding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# 构建一个多层卷积网络

我们尝试使用tensorflow的低级API构建一个CNN，来对cifar10进行分类。

In [ ]:
import tensorflow as tf
sess = tf.InteractiveSession()

# 权重初始化

为了创建这个模型，我们需要创建大量的权重和偏置项。

这个模型中的权重在初始化时应该加入少量的噪声来打破对称性以及避免0梯度。

由于我们使用的是ReLU神经元，因此比较好的做法是用一个较小的正数来初始化偏置项，以避免神经元节点输出恒为0的问题（dead neurons）。

为了不在建立模型的时候反复做初始化操作，我们定义两个函数用于初始化。

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# 卷积和池化

TensorFlow在卷积和池化上有很强的灵活性。我们怎么处理边界？步长应该设多大？

在这个实例里，我们会一直使用vanilla版本。我们的卷积使用1步长（stride size），0边距（padding size）的模板，保证输出和输入是同一个大小。

我们的池化用简单传统的2x2大小的模板做max pooling。

为了代码更简洁，我们把这部分抽象成一个函数。

In [ ]:
# w为输入
# W为全集核
def conv2d(x, W):
    # strides在官方定义中是一个一维具有四个元素的张量，其规定前后必须为1，所以我们可以改的是中间两个数，
    # 中间两个数分别代表了水平滑动和垂直滑动步长值。
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 占位符
我们通过为输入图像和目标输出类别创建节点，来开始构建计算图。

In [ ]:
# x为输入的数据，第一个维度为None
x = tf.placeholder("float", shape=[None, 32, 32, 3])
# y_为输入数据对应的label
y_ = tf.placeholder("float", shape=[None, 10])

# 第一层卷积
现在我们可以开始实现第一层了。

它由一个卷积接一个max pooling完成。

卷积在每个5x5的patch中算出32个特征。

卷积的权重张量形状是[5, 5, 3, 32]，前两个维度是patch的大小，接着是输入的通道数目，最后是输出的通道数目。

而对于每一个输出通道都有一个对应的偏置量。

In [ ]:
W_conv1 = weight_variable([5, 5, 3, 32])
b_conv1 = bias_variable([32])

In [ ]:
h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# 第二层卷积
为了构建一个更深的网络，我们会把几个类似的层堆叠起来。第二层中，每个5x5的patch会得到64个特征。

In [ ]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# 全连接层
现在，图片尺寸减小到7x7，我们加入一个有1024个神经元的全连接层，用于处理整个图片。我们把池化层输出的张量reshape成一些向量，乘上权重矩阵，加上偏置，然后对其使用ReLU。

In [ ]:
W_fc1 = weight_variable([8 * 8 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 8 * 8 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
为了减少过拟合，我们在输出层之前加入dropout。
我们用一个placeholder来代表一个神经元的输出在dropout中保持不变的概率。
这样我们可以在训练过程中启用dropout，在测试过程中关闭dropout。 
TensorFlow的tf.nn.dropout操作除了可以屏蔽神经元的输出外，还会自动处理神经元输出值的scale。所以用dropout的时候可以不用考虑scale。

In [ ]:
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, rate = 1-keep_prob)

# 输出层
最后，我们添加一个softmax层，就像前面的单层softmax regression一样。

In [ ]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


# 训练和评估模型
这个模型的效果如何呢？

我们仍然使用softmax作为输出。

ADAM优化器来做梯度最速下降，

在feed_dict中加入额外的参数keep_prob来控制dropout比例。

然后每100次迭代输出一次日志。

In [ ]:
cross_entropy_map = tf.losses.softmax_cross_entropy(onehot_labels=y_, logits=y)
train_step = tf.train.GradientDescentOptimizer(1e-4).minimize(cross_entropy_map)
# 如果相等则返回True，不想等则返回False
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
# 计算平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.global_variables_initializer())

In [ ]:
assert x_train.shape[0] == y_train.shape[0]

In [ ]:
features_placeholder = tf.placeholder(x_train.dtype, x_train.shape)
labels_placeholder = tf.placeholder(y_train.dtype, y_train.shape)
dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))

In [ ]:
dataset = dataset.shuffle(buffer_size=1000)
dataset = dataset.repeat(3)
dataset = dataset.batch(128)

In [ ]:
iterator = dataset.make_initializable_iterator()
sess.run(iterator.initializer, feed_dict={features_placeholder: x_train, labels_placeholder: y_train})

In [ ]:
next_element = iterator.get_next()

In [ ]:
import numpy as np

step = 1
try:
    while True:
        batch = sess.run(next_element, feed_dict={features_placeholder: x_train, labels_placeholder: y_train})
        features = batch[0]
        label = batch[1]
        print("features' shape is", features.shape)
        train_accuracy = accuracy.eval(feed_dict={x: features , y_: label, keep_prob: 1.0})

        sess.run(train_step, feed_dict={x: features, y_: label, keep_prob: 0.5})
        
        step = step + 1
        if step % 200 == 0:
            print("\nstep %d, training accuracy %f " % (step, train_accuracy))
        elif step % 20 == 0:
            print('.', end='')
        
except tf.errors.OutOfRangeError:
    print("End of dataset")
print(step)

In [ ]:
features = x_test
label = y_test

In [ ]:
test_accuracy = accuracy.eval(feed_dict={x: features , y_: label, keep_prob: 1.0})

In [ ]:
test_accuracy

目前为止，我们已经学会了用TensorFlow低级api中tf.nn api 搭建、训练和评估一个复杂一点儿的深度学习模型。